# ZnTrack v0.3 Migration Guide

The new ZnTrack release v0.3 includes some restructures that are not backwards compatible.
The reasons for these changes to the API are justified by better user experience, easier testing and more pythonic code.
The following Guide will show how easy a prior ZnTrack setup can be migrated to ZnTrack v0.3.

The main changes include

- class inheritance replaces the `@Node` decorator
- the `__call__` is no longer required to write the dvc files and can be replaced by `Node.write_graph()`
- Loading a Node is now a classmethod `Node.load(name=<my_name>)`
- `dvc.results` is replaced by `zn.outs` to seperate `zntrack.dvc` and `zntrack.zn`
- `dvc.params` is depreciated and replaced by `zn.params` (it will be loaded, `dvc.<option>` is almost strictly related to file paths)

Let's have a look at some examples:

In [1]:
from zntrack import Node, dvc, zn


@Node()  # replaced by class inheritance
class HelloWorld:
    parameter = dvc.params()  # replaced by zn.params()
    outs = dvc.result()  # replaced by zn.outs()

    def __call__(self, param):  # can still be used, but must follow <node>.write_dvc
        self.parameter = param

    def run(self):
        self.outs = self.parameter


# Writing the DVC File:
HelloWorld()(param=123)

# Loading the Node
hello_world = HelloWorld(load=True)

This class would now look like

In [ ]:
class HelloWorld(Node):
    parameter = zn.params()
    outs = zn.outs()

    def __init__(self, param=None, **kwargs):
        # every argument must default to something
        super().__init__(**kwargs)
        self.parameter = param
        if not self.is_loaded:
            # some conditions in the init only work if the stage is not loaded!
            # otherwise, it could be, that param=None.
            if param > 10:
                self.parameter = 10

    def run(self):
        self.outs = self.parameter


# Writing the DVC File:
HelloWorld(param=123).write_graph()

# Loading the Node
hello_world = HelloWorld.load()

Arguments that were previously passed through the decorator are now moved to the `super().__init__()`

In [ ]:
@Node(always_changed=True)
class HelloWorld:
    pass


# The extra parameters that were previously placed in the @Node
# are now passed through kwargs (name) or through `write_graph()`


class HelloWorld(Node):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)


# writing the graph with extra arguments
HelloWorld(name="MyName").write_graph(
    always_changed=True,
    external=True,
    no_commit=True,
    no_run_cache=True,
    no_exec=False,
    force=False,
)

# Loading a named Node
HelloWorld.load(name="MyName")